# Predicting NBA Scores using Machine Learning

### George Tao,  Jeremiah Sahabu, Isaac Chau

#### Introduction

Sports inheritantly has large variability. Thus, we wanted to see if we can predict which team will win through supervised Machine Learning. We decided to use Logistic Regression for our models as we believe it fits the scenario best.

#### Dataset:

#### Features:

We saw a lot of features given ranging from player stats, scores, among others. We used a combination of these features as well as features we created from the dataset above to feed into our model. See list below of features we identified significant

- Consecutive Away Games
- Wins in the last 10 Games
- Officials 
- Players
- Back to Back Games
- eFG% (season average)
- eFG% allowed (season average)
- Team Fouls (season average)
- Turnovers 
- Offensive Rebounds



### Loading in the Data

### Parsing the Data

### Models & Plots

### Conclusion